In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np

## 0. DataFrame

In [2]:
football_df = pd.read_csv('data/all_data_with_elo.csv', low_memory = False)
football_df

,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,...,B365D,B365A,IWH,IWD,IWA,WHH,WHD,WHA,HomeTeamELO,AwayTeamELO
0,0,F1,2002-08-02,Guingamp,Lyon,3.0,3.0,D,NaN,NaN,...,3.000,2.100,3.0,2.9,2.2,NaN,NaN,NaN,1576.646240,1771.473022
1,1,F1,2002-08-03,Bastia,Lens,1.0,1.0,D,NaN,NaN,...,2.875,2.625,2.6,2.8,2.6,2.50,3.1,2.50,1639.578369,1723.067871
2,2,F1,2002-08-03,Lille,Bordeaux,0.0,3.0,A,NaN,NaN,...,3.000,3.200,2.2,2.8,3.1,2.20,3.1,2.87,1698.813965,1707.882446
3,3,F1,2002-08-03,Marseille,Nantes,0.0,2.0,A,NaN,NaN,...,3.000,2.875,2.2,2.9,3.0,2.20,3.0,3.00,1636.971436,1708.669556
4,4,F1,2002-08-03,Montpellier,Rennes,1.0,0.0,H,NaN,NaN,...,3.000,4.000,1.9,2.9,3.8,1.90,3.1,3.60,1626.294678,1638.674683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38975,38975,SP1,2024-05-26,Las Palmas,Alaves,1.0,1.0,D,10.0,14.0,...,3.300,3.000,NaN,NaN,NaN,2.40,3.4,2.88,1586.205078,1643.290894
38976,38976,I1,2024-05-26,Lazio,Sassuolo,1.0,1.0,D,8.0,8.0,...,5.250,7.500,NaN,NaN,NaN,1.33,5.5,8.00,1788.179199,1599.679199
38977,38977,I1,2024-05-26,Napoli,Lecce,0.0,0.0,D,24.0,10.0,...,5.000,6.250,NaN,NaN,NaN,1.40,4.8,7.50,1782.281006,1621.917236
38978,38978,SP1,2024-05-26,Sevilla,Barcelona,1.0,2.0,A,15.0,11.0,...,4.000,1.800,NaN,NaN,NaN,3.80,4.2,1.80,1673.872314,1879.718994


## 1. Descriptive Statistics 

**1.1 DataFrame Shape**

In [3]:
# no. rows and no. cols
football_df.shape

(38980, 23)

In [4]:
# feature names
print(football_df.columns.tolist())

['Unnamed: 0', 'Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HS', 'AS', 'HST', 'AST', 'B365H', 'B365D', 'B365A', 'IWH', 'IWD', 'IWA', 'WHH', 'WHD', 'WHA', 'HomeTeamELO', 'AwayTeamELO']


**1.2 NaN Values**

In [5]:
football_df.isnull().sum()

Unnamed: 0        0
Div               0
Date              0
HomeTeam          0
AwayTeam          0
FTHG              0
FTAG              0
FTR               0
HS             2929
AS             2929
HST            3735
AST            3735
B365H            57
B365D            57
B365A            57
IWH            1015
IWD            1015
IWA            1015
WHH             588
WHD             588
WHA             588
HomeTeamELO     273
AwayTeamELO     276
dtype: int64

In [6]:
# total elements in 
football_df.size

896540

In [7]:
# total number of NaN
football_df.size - football_df.count().sum()

18857

In [8]:
# total number of NaN rows
football_df.isnull().any(axis = 1).sum()

5586

In [9]:
# total number of NaN columns
football_df.isnull().any(axis = 0).sum()

15

## 2. Data Wrangling and Feature Transformation/Development

### 2.1 NaN Handling

`TODO`: drop NaN values along columns: {Date, Home Team, Away Team, FTR} <br>
`TODO`: identify betting odds w/ most available data

In [10]:
# 当前方法仅提取这几个字段 分区 日期 主队 客队 full-time-result 三家机构的胜平负 主队ELO评分 客队ELO评分
nan_mask = ['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTR', 'B365H', 'B365D', 'B365A', 
            'IWH', 'IWD', 'IWA', 'WHH', 'WHD', 'WHA', 'HomeTeamELO', 'AwayTeamELO']

In [11]:
# 删除指定列中含有缺失值的行
#football_df.FTR.replace('nan', np.nan, inplace=True)
nan_football_df = football_df.dropna(subset = nan_mask)
nan_football_df

,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,...,B365D,B365A,IWH,IWD,IWA,WHH,WHD,WHA,HomeTeamELO,AwayTeamELO
1,1,F1,2002-08-03,Bastia,Lens,1.0,1.0,D,NaN,NaN,...,2.875,2.625,2.60,2.80,2.60,2.50,3.1,2.50,1639.578369,1723.067871
2,2,F1,2002-08-03,Lille,Bordeaux,0.0,3.0,A,NaN,NaN,...,3.000,3.200,2.20,2.80,3.10,2.20,3.1,2.87,1698.813965,1707.882446
3,3,F1,2002-08-03,Marseille,Nantes,0.0,2.0,A,NaN,NaN,...,3.000,2.875,2.20,2.90,3.00,2.20,3.0,3.00,1636.971436,1708.669556
4,4,F1,2002-08-03,Montpellier,Rennes,1.0,0.0,H,NaN,NaN,...,3.000,4.000,1.90,2.90,3.80,1.90,3.1,3.60,1626.294678,1638.674683
5,5,F1,2002-08-03,Nice,Le Havre,1.0,2.0,A,NaN,NaN,...,3.000,3.750,2.00,2.90,3.40,1.83,3.2,3.75,1529.394897,1571.807251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38088,38088,SP1,2024-01-03,Granada,Cadiz,2.0,0.0,H,15.0,15.0,...,3.100,3.500,2.25,3.20,3.35,2.15,3.2,3.40,1574.769897,1628.365479
38089,38089,SP1,2024-01-03,Real Madrid,Mallorca,1.0,0.0,H,17.0,7.0,...,6.500,15.000,1.22,6.50,13.00,1.20,6.0,13.00,1968.716064,1645.844849
38090,38090,SP1,2024-01-04,Las Palmas,Barcelona,1.0,2.0,A,12.0,15.0,...,4.330,1.570,5.50,4.30,1.57,5.50,4.2,1.53,1629.086792,1863.405640
38091,38091,SP1,2024-01-04,Osasuna,Almeria,1.0,0.0,H,14.0,4.0,...,4.000,4.750,1.70,3.90,4.80,1.65,3.9,4.60,1653.191528,1550.888062


In [12]:
# resize shape
football_df.shape[0] - nan_football_df.shape[0]

2130

### 2.2 Feature Encoding <br>
* $\phi(Date)$ $\Rightarrow$ one column for *year*, second column for *month*, third column for *day of year*
* One hot encode Division, Home and Away Teams
* Label encode Full Time Result (Win/Draw/Loss)

In [13]:
feats = nan_mask

In [14]:
nan_mask

['Div',
 'Date',
 'HomeTeam',
 'AwayTeam',
 'FTR',
 'B365H',
 'B365D',
 'B365A',
 'IWH',
 'IWD',
 'IWA',
 'WHH',
 'WHD',
 'WHA',
 'HomeTeamELO',
 'AwayTeamELO']

In [15]:
nan_football_df

,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,...,B365D,B365A,IWH,IWD,IWA,WHH,WHD,WHA,HomeTeamELO,AwayTeamELO
1,1,F1,2002-08-03,Bastia,Lens,1.0,1.0,D,NaN,NaN,...,2.875,2.625,2.60,2.80,2.60,2.50,3.1,2.50,1639.578369,1723.067871
2,2,F1,2002-08-03,Lille,Bordeaux,0.0,3.0,A,NaN,NaN,...,3.000,3.200,2.20,2.80,3.10,2.20,3.1,2.87,1698.813965,1707.882446
3,3,F1,2002-08-03,Marseille,Nantes,0.0,2.0,A,NaN,NaN,...,3.000,2.875,2.20,2.90,3.00,2.20,3.0,3.00,1636.971436,1708.669556
4,4,F1,2002-08-03,Montpellier,Rennes,1.0,0.0,H,NaN,NaN,...,3.000,4.000,1.90,2.90,3.80,1.90,3.1,3.60,1626.294678,1638.674683
5,5,F1,2002-08-03,Nice,Le Havre,1.0,2.0,A,NaN,NaN,...,3.000,3.750,2.00,2.90,3.40,1.83,3.2,3.75,1529.394897,1571.807251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38088,38088,SP1,2024-01-03,Granada,Cadiz,2.0,0.0,H,15.0,15.0,...,3.100,3.500,2.25,3.20,3.35,2.15,3.2,3.40,1574.769897,1628.365479
38089,38089,SP1,2024-01-03,Real Madrid,Mallorca,1.0,0.0,H,17.0,7.0,...,6.500,15.000,1.22,6.50,13.00,1.20,6.0,13.00,1968.716064,1645.844849
38090,38090,SP1,2024-01-04,Las Palmas,Barcelona,1.0,2.0,A,12.0,15.0,...,4.330,1.570,5.50,4.30,1.57,5.50,4.2,1.53,1629.086792,1863.405640
38091,38091,SP1,2024-01-04,Osasuna,Almeria,1.0,0.0,H,14.0,4.0,...,4.000,4.750,1.70,3.90,4.80,1.65,3.9,4.60,1653.191528,1550.888062


In [16]:
learning_df = nan_football_df.copy()[feats]
learning_df

,Div,Date,HomeTeam,AwayTeam,FTR,B365H,B365D,B365A,IWH,IWD,IWA,WHH,WHD,WHA,HomeTeamELO,AwayTeamELO
1,F1,2002-08-03,Bastia,Lens,D,2.500,2.875,2.625,2.60,2.80,2.60,2.50,3.1,2.50,1639.578369,1723.067871
2,F1,2002-08-03,Lille,Bordeaux,A,2.100,3.000,3.200,2.20,2.80,3.10,2.20,3.1,2.87,1698.813965,1707.882446
3,F1,2002-08-03,Marseille,Nantes,A,2.250,3.000,2.875,2.20,2.90,3.00,2.20,3.0,3.00,1636.971436,1708.669556
4,F1,2002-08-03,Montpellier,Rennes,H,1.833,3.000,4.000,1.90,2.90,3.80,1.90,3.1,3.60,1626.294678,1638.674683
5,F1,2002-08-03,Nice,Le Havre,A,1.909,3.000,3.750,2.00,2.90,3.40,1.83,3.2,3.75,1529.394897,1571.807251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38088,SP1,2024-01-03,Granada,Cadiz,H,2.250,3.100,3.500,2.25,3.20,3.35,2.15,3.2,3.40,1574.769897,1628.365479
38089,SP1,2024-01-03,Real Madrid,Mallorca,H,1.200,6.500,15.000,1.22,6.50,13.00,1.20,6.0,13.00,1968.716064,1645.844849
38090,SP1,2024-01-04,Las Palmas,Barcelona,A,5.500,4.330,1.570,5.50,4.30,1.57,5.50,4.2,1.53,1629.086792,1863.405640
38091,SP1,2024-01-04,Osasuna,Almeria,H,1.700,4.000,4.750,1.70,3.90,4.80,1.65,3.9,4.60,1653.191528,1550.888062


In [17]:
learning_df.reset_index(inplace=True, drop=True)
learning_df

,Div,Date,HomeTeam,AwayTeam,FTR,B365H,B365D,B365A,IWH,IWD,IWA,WHH,WHD,WHA,HomeTeamELO,AwayTeamELO
0,F1,2002-08-03,Bastia,Lens,D,2.500,2.875,2.625,2.60,2.80,2.60,2.50,3.1,2.50,1639.578369,1723.067871
1,F1,2002-08-03,Lille,Bordeaux,A,2.100,3.000,3.200,2.20,2.80,3.10,2.20,3.1,2.87,1698.813965,1707.882446
2,F1,2002-08-03,Marseille,Nantes,A,2.250,3.000,2.875,2.20,2.90,3.00,2.20,3.0,3.00,1636.971436,1708.669556
3,F1,2002-08-03,Montpellier,Rennes,H,1.833,3.000,4.000,1.90,2.90,3.80,1.90,3.1,3.60,1626.294678,1638.674683
4,F1,2002-08-03,Nice,Le Havre,A,1.909,3.000,3.750,2.00,2.90,3.40,1.83,3.2,3.75,1529.394897,1571.807251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36845,SP1,2024-01-03,Granada,Cadiz,H,2.250,3.100,3.500,2.25,3.20,3.35,2.15,3.2,3.40,1574.769897,1628.365479
36846,SP1,2024-01-03,Real Madrid,Mallorca,H,1.200,6.500,15.000,1.22,6.50,13.00,1.20,6.0,13.00,1968.716064,1645.844849
36847,SP1,2024-01-04,Las Palmas,Barcelona,A,5.500,4.330,1.570,5.50,4.30,1.57,5.50,4.2,1.53,1629.086792,1863.405640
36848,SP1,2024-01-04,Osasuna,Almeria,H,1.700,4.000,4.750,1.70,3.90,4.80,1.65,3.9,4.60,1653.191528,1550.888062


**2.2.1 Division and Home/Away Team Encoding**

In [18]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

div_encoder = OneHotEncoder()
home_encoder = OneHotEncoder()
away_encoder = OneHotEncoder()

In [19]:
onehot_div = div_encoder.fit_transform(learning_df.Div.values.reshape(-1,1)).toarray().astype(int)
onehot_div_df = pd.DataFrame(onehot_div, columns = ["Div "+str(int(i)) for i in range(onehot_div.shape[1])])

onehot_home = home_encoder.fit_transform(learning_df.HomeTeam.values.reshape(-1,1)).toarray().astype(int)
onehot_home_df = pd.DataFrame(onehot_home, columns = ['HomeTeam ' + str(int(i)) for i in np.arange(onehot_home.shape[1])])

onehot_away = away_encoder.fit_transform(learning_df.AwayTeam.values.reshape(-1,1)).toarray().astype(int)
onehot_away_df = pd.DataFrame(onehot_away, columns = ['AwayTeam ' + str(int(i)) for i in np.arange(onehot_away.shape[1])])

In [20]:
learning_df = pd.concat([learning_df, onehot_div_df, onehot_home_df, onehot_away_df], axis = 1)
learning_df.drop(columns = ['Div'], inplace = True)

In [21]:
learning_df

,Date,HomeTeam,AwayTeam,FTR,B365H,B365D,B365A,IWH,IWD,IWA,...,AwayTeam 194,AwayTeam 195,AwayTeam 196,AwayTeam 197,AwayTeam 198,AwayTeam 199,AwayTeam 200,AwayTeam 201,AwayTeam 202,AwayTeam 203
0,2002-08-03,Bastia,Lens,D,2.500,2.875,2.625,2.60,2.80,2.60,...,0,0,0,0,0,0,0,0,0,0
1,2002-08-03,Lille,Bordeaux,A,2.100,3.000,3.200,2.20,2.80,3.10,...,0,0,0,0,0,0,0,0,0,0
2,2002-08-03,Marseille,Nantes,A,2.250,3.000,2.875,2.20,2.90,3.00,...,0,0,0,0,0,0,0,0,0,0
3,2002-08-03,Montpellier,Rennes,H,1.833,3.000,4.000,1.90,2.90,3.80,...,0,0,0,0,0,0,0,0,0,0
4,2002-08-03,Nice,Le Havre,A,1.909,3.000,3.750,2.00,2.90,3.40,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36845,2024-01-03,Granada,Cadiz,H,2.250,3.100,3.500,2.25,3.20,3.35,...,0,0,0,0,0,0,0,0,0,0
36846,2024-01-03,Real Madrid,Mallorca,H,1.200,6.500,15.000,1.22,6.50,13.00,...,0,0,0,0,0,0,0,0,0,0
36847,2024-01-04,Las Palmas,Barcelona,A,5.500,4.330,1.570,5.50,4.30,1.57,...,0,0,0,0,0,0,0,0,0,0
36848,2024-01-04,Osasuna,Almeria,H,1.700,4.000,4.750,1.70,3.90,4.80,...,0,0,0,0,0,0,0,0,0,0


**2.2.2 Full Time Result Encoding**

In [22]:
target_encoder = LabelEncoder()
learning_df['Result'] = target_encoder.fit_transform(learning_df.FTR) 

**2.2.3 Date Encoding**

In [23]:
learning_df['Year'] = pd.DatetimeIndex(learning_df.Date).year

learning_df['Month'] = pd.DatetimeIndex(learning_df.Date).month
learning_df['Sin_Month'] = np.sin(2*np.pi*learning_df.Month/12)
learning_df['Cos_Month'] = np.cos(2*np.pi*learning_df.Month/12)

learning_df['DayofYear'] = pd.DatetimeIndex(learning_df.Date).dayofyear
learning_df['Sin_Day'] = np.sin(2*np.pi*learning_df.DayofYear/365)
learning_df['Cos_Day'] = np.cos(2*np.pi*learning_df.DayofYear/365)

learning_df.drop(columns = ['Date','Month'], inplace = True)
# learning_df.drop(columns = ['Date'], inplace = True)

In [24]:
learning_df

,HomeTeam,AwayTeam,FTR,B365H,B365D,B365A,IWH,IWD,IWA,WHH,...,AwayTeam 201,AwayTeam 202,AwayTeam 203,Result,Year,Sin_Month,Cos_Month,DayofYear,Sin_Day,Cos_Day
0,Bastia,Lens,D,2.500,2.875,2.625,2.60,2.80,2.60,2.50,...,0,0,0,1,2002,-0.866025,-0.500000,215,-0.530730,-0.847541
1,Lille,Bordeaux,A,2.100,3.000,3.200,2.20,2.80,3.10,2.20,...,0,0,0,0,2002,-0.866025,-0.500000,215,-0.530730,-0.847541
2,Marseille,Nantes,A,2.250,3.000,2.875,2.20,2.90,3.00,2.20,...,0,0,0,0,2002,-0.866025,-0.500000,215,-0.530730,-0.847541
3,Montpellier,Rennes,H,1.833,3.000,4.000,1.90,2.90,3.80,1.90,...,0,0,0,2,2002,-0.866025,-0.500000,215,-0.530730,-0.847541
4,Nice,Le Havre,A,1.909,3.000,3.750,2.00,2.90,3.40,1.83,...,0,0,0,0,2002,-0.866025,-0.500000,215,-0.530730,-0.847541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36845,Granada,Cadiz,H,2.250,3.100,3.500,2.25,3.20,3.35,2.15,...,0,0,0,2,2024,0.500000,0.866025,3,0.051620,0.998667
36846,Real Madrid,Mallorca,H,1.200,6.500,15.000,1.22,6.50,13.00,1.20,...,0,0,0,2,2024,0.500000,0.866025,3,0.051620,0.998667
36847,Las Palmas,Barcelona,A,5.500,4.330,1.570,5.50,4.30,1.57,5.50,...,0,0,0,0,2024,0.500000,0.866025,4,0.068802,0.997630
36848,Osasuna,Almeria,H,1.700,4.000,4.750,1.70,3.90,4.80,1.65,...,0,0,0,2,2024,0.500000,0.866025,4,0.068802,0.997630


### 2.3 Feature Engineering <br>
* $\phi(x)$ feature transformation $\Rightarrow$ last match result, win/loss streak to date, wins to season date
* $\phi(x)$ feature engineering $\Rightarrow$ average the home, away, and draw odds

**2.3.1 Last Match Result** <br>
Indicate the result from the last match played between both teams

In [25]:
# 定义一个函数来计算两队之间上一场比赛的结果
def compute_lastmatchres(df):
    
    unique_matchups = list(set((list(zip(df.HomeTeam, df.AwayTeam)))))
    df['Last Match Result'] = np.nan
    for home, away in unique_matchups:
        matchup_df = df[(df.HomeTeam == home) & (df.AwayTeam == away)]
        # 使用 shift(1) 方法将 FTR（全场比赛结果）列中的数据向下移动一行，这样每行的 last_match_result 将对应于这两队之前的一场比赛的结果。fill_value='Na' 确保了数据移动后空出的位置填充为 'Na'。
        last_match_result = matchup_df.FTR.shift(1, fill_value='Na')
        df.loc[matchup_df.index, 'Last Match Result'] = last_match_result
        
    lmr_encoder = LabelEncoder()
    df['Last Match Result'] = lmr_encoder.fit_transform(df['Last Match Result'])
    df.drop(columns = ['FTR'], inplace = True)
    return df

In [26]:
learning_df = compute_lastmatchres(learning_df)
# learning_df.drop(columns = ['FTR'], inplace = True)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_28336\1506541586.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Na' 'D']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[matchup_df.index, 'Last Match Result'] = last_match_result


**2.3.2 Home and Away Win/Loss Streak** <br>
Important note about this feature: the win/loss streak is the teams *home* and *away* win streak, *not* its ***consecutive*** win/loss streak.

In [27]:
# https://stackoverflow.com/questions/52976336/compute-winning-streak-with-pandas
# https://joshdevlin.com/blog/calculate-streaks-in-pandas/

In [28]:
def compute_winstreak(df):
    
    years = df.Year.unique()
    df_lst = []    
    for year in years:
        
        year_df = df[df.Year == year]
        year_df['HomeWin'] = year_df.Result.replace([0, 1, 2], [0, 0, 1])
        year_df['AwayWin'] = year_df.Result.replace([0, 1, 2], [1, 0, 0])
        year_df['HomeWinStreak'] = None
        year_df['AwayWinStreak'] = None
        
        hometeams = year_df.HomeTeam.unique()
        awayteams = year_df.AwayTeam.unique()
        
        for team in hometeams:
            team_df = year_df[(year_df.HomeTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))

            team_grouper = (team_df.HomeWin != team_df.HomeWin.shift()).cumsum()
            team_df['HomeWinStreak'] = team_df[['HomeWin']].groupby(team_grouper).cumsum()
            team_df.loc[team_df.HomeWinStreak >0, 'HomeWinStreak'] -= 1
            year_df.loc[team_df.index, 'HomeWinStreak'] = team_df.HomeWinStreak
            
        for team in awayteams:
            team_df = year_df[(year_df.AwayTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))

            team_grouper = (team_df.AwayWin != team_df.AwayWin.shift()).cumsum()
            team_df['AwayWinStreak'] = team_df[['AwayWin']].groupby(team_grouper).cumsum()
            team_df.loc[team_df.AwayWinStreak >0, 'AwayWinStreak'] -= 1
            year_df.loc[team_df.index, 'AwayWinStreak'] = team_df.AwayWinStreak
            
        df_lst.append(year_df)
        
    return pd.concat(df_lst, axis = 0).drop(columns = ['HomeWin', 'AwayWin'])#,'DayofYear'])

In [29]:
learning_df = compute_winstreak(learning_df)

**2.3.4 Season Home/Away Wins to Date** <br>
Indicate the number of wins for a team as home and away to date within current season

In [30]:
toy = learning_df[(learning_df.Year == 2010) & (learning_df.HomeTeam == 'Barcelona')][['HomeTeam', 'AwayTeam', 'Result']]
toy['HomeWin'] = toy.Result.replace([0, 1, 2], [0, 0, 1])
toy['HomeWinsToDate'] = toy.HomeWin.cumsum()

In [31]:
def compute_winstodate(df):
    
    years = df.Year.unique()
    df_lst = []    
    for year in years:
        
        year_df = df[df.Year == year]
        year_df['HomeWin'] = year_df.Result.replace([0, 1, 2], [0, 0, 1])
        year_df['AwayWin'] = year_df.Result.replace([0, 1, 2], [1, 0, 0])
        year_df['HomeWinsToDate'] = None
        year_df['AwayWinsToDate'] = None
        
        hometeams = year_df.HomeTeam.unique()
        awayteams = year_df.AwayTeam.unique()
        
        for team in hometeams:
            team_df = year_df[(year_df.HomeTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))

            team_df['HomeWinsToDate'] = team_df.HomeWin.cumsum()
            year_df.loc[team_df.index, 'HomeWinsToDate'] = team_df.HomeWinsToDate
            
        for team in awayteams:
            team_df = year_df[(year_df.AwayTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))
            
            team_df['AwayWinsToDate'] = team_df.AwayWin.cumsum()
            year_df.loc[team_df.index, 'AwayWinsToDate'] = team_df.AwayWinsToDate
            
        df_lst.append(year_df)
        
    return pd.concat(df_lst, axis = 0).drop(columns = ['HomeWin', 'AwayWin','DayofYear'])

In [32]:
learning_df = compute_winstodate(learning_df)
learning_df.drop(columns = ['HomeTeam', 'AwayTeam'], inplace = True)

**2.3.5 Website Odds** <br>
The `betting odds` recorded by various betting websites offer insight into sentiment surrounding the outcome of a particular game. 

In [33]:
betting_feats = ['B365H', 'B365D', 'B365A', 'IWH', 'IWD', 'IWA', 'WHH', 'WHD', 'WHA']
betting_feats

['B365H', 'B365D', 'B365A', 'IWH', 'IWD', 'IWA', 'WHH', 'WHD', 'WHA']

In [34]:
def compute_meanodds(df, betting_feats):
    """
    """
    home_odds = []
    away_odds = []
    draw_odds = []
    for odd in betting_feats:
        odd_type = odd[-1]
        if odd_type == 'H':
            home_odds.append(odd)
        elif odd_type == 'A':
            away_odds.append(odd)
        else:
            draw_odds.append(odd)
    avg_home_odds = df[home_odds].mean(axis=1)
    avg_away_odds = df[away_odds].mean(axis=1)
    avg_draw_odds = df[draw_odds].mean(axis=1)
    
    ordered_cols = ['HomeOdds', 'AwayOdds', 'DrawOdds'] + df.columns.tolist()
    
    df['HomeOdds'] = avg_home_odds
    df['AwayOdds'] = avg_away_odds
    df['DrawOdds'] = avg_draw_odds
    
    return df[ordered_cols]

In [35]:
learning_df = compute_meanodds(learning_df, betting_feats)

### 2.4 Peek @ Learning DataFrame

In [36]:
learning_df

,HomeOdds,AwayOdds,DrawOdds,B365H,B365D,B365A,IWH,IWD,IWA,WHH,...,Year,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak,HomeWinsToDate,AwayWinsToDate
0,2.533333,2.575000,2.925000,2.500,2.875,2.625,2.60,2.80,2.60,2.50,...,2002,-0.866025,-0.500000,-0.530730,-0.847541,3,0,0,0,0
1,2.166667,3.056667,2.966667,2.100,3.000,3.200,2.20,2.80,3.10,2.20,...,2002,-0.866025,-0.500000,-0.530730,-0.847541,3,0,0,0,1
2,2.216667,2.958333,2.966667,2.250,3.000,2.875,2.20,2.90,3.00,2.20,...,2002,-0.866025,-0.500000,-0.530730,-0.847541,3,0,0,0,1
3,1.877667,3.800000,3.000000,1.833,3.000,4.000,1.90,2.90,3.80,1.90,...,2002,-0.866025,-0.500000,-0.530730,-0.847541,3,0,0,1,0
4,1.913000,3.633333,3.033333,1.909,3.000,3.750,2.00,2.90,3.40,1.83,...,2002,-0.866025,-0.500000,-0.530730,-0.847541,3,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36845,2.216667,3.416667,3.166667,2.250,3.100,3.500,2.25,3.20,3.35,2.15,...,2024,0.500000,0.866025,0.051620,0.998667,1,0,0,1,0
36846,1.206667,13.666667,6.333333,1.200,6.500,15.000,1.22,6.50,13.00,1.20,...,2024,0.500000,0.866025,0.051620,0.998667,2,0,0,1,0
36847,5.500000,1.556667,4.276667,5.500,4.330,1.570,5.50,4.30,1.57,5.50,...,2024,0.500000,0.866025,0.068802,0.997630,1,0,0,0,1
36848,1.683333,4.716667,3.933333,1.700,4.000,4.750,1.70,3.90,4.80,1.65,...,2024,0.500000,0.866025,0.068802,0.997630,2,0,0,1,0


# 3. Model Development

* Establish a baseline Logistic Regression model fit over the entire learning dataframe without special regard to *division* and *team*. 
* Train model over 16 seasons, and predict for the remaining 3 seasons (approximate 80-20 split)

### 3.1 Train and Test Split

In [37]:
split = 0.80
no_seasons = 20

print('No. seasons to train over: ' + str(round(split*no_seasons)))

No. seasons to train over: 16


In [38]:
X, y = learning_df.loc[:, learning_df.columns != 'Result'], learning_df[['Result']]

In [39]:
# full_feat = ['HomeWinStreak','AwayWinStreak','HomeWinsToDate', 'AwayWinsToDate', 'Last Match Result',
#              'HomeTeamELO', 'AwayTeamELO', 'HomeOdds', 'AwayOdds', 'DrawOdds'] + betting_feats

# exclude_feats = ['HomeWinsToDate', 'AwayWinsToDate', 'Last Match Result'] 

In [40]:
# X = X[X.columns[~X.columns.isin(exclude_feats)]]
# X

In [41]:
X

,HomeOdds,AwayOdds,DrawOdds,B365H,B365D,B365A,IWH,IWD,IWA,WHH,...,Year,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak,HomeWinsToDate,AwayWinsToDate
0,2.533333,2.575000,2.925000,2.500,2.875,2.625,2.60,2.80,2.60,2.50,...,2002,-0.866025,-0.500000,-0.530730,-0.847541,3,0,0,0,0
1,2.166667,3.056667,2.966667,2.100,3.000,3.200,2.20,2.80,3.10,2.20,...,2002,-0.866025,-0.500000,-0.530730,-0.847541,3,0,0,0,1
2,2.216667,2.958333,2.966667,2.250,3.000,2.875,2.20,2.90,3.00,2.20,...,2002,-0.866025,-0.500000,-0.530730,-0.847541,3,0,0,0,1
3,1.877667,3.800000,3.000000,1.833,3.000,4.000,1.90,2.90,3.80,1.90,...,2002,-0.866025,-0.500000,-0.530730,-0.847541,3,0,0,1,0
4,1.913000,3.633333,3.033333,1.909,3.000,3.750,2.00,2.90,3.40,1.83,...,2002,-0.866025,-0.500000,-0.530730,-0.847541,3,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36845,2.216667,3.416667,3.166667,2.250,3.100,3.500,2.25,3.20,3.35,2.15,...,2024,0.500000,0.866025,0.051620,0.998667,1,0,0,1,0
36846,1.206667,13.666667,6.333333,1.200,6.500,15.000,1.22,6.50,13.00,1.20,...,2024,0.500000,0.866025,0.051620,0.998667,2,0,0,1,0
36847,5.500000,1.556667,4.276667,5.500,4.330,1.570,5.50,4.30,1.57,5.50,...,2024,0.500000,0.866025,0.068802,0.997630,1,0,0,0,1
36848,1.683333,4.716667,3.933333,1.700,4.000,4.750,1.70,3.90,4.80,1.65,...,2024,0.500000,0.866025,0.068802,0.997630,2,0,0,1,0


In [42]:
y

,Result
0,1
1,0
2,0
3,2
4,0
...,...
36845,2
36846,2
36847,0
36848,2


In [43]:
split_year = 2020

In [44]:
# 切分训练集和测试集
xTr, xTe = X[X.Year <= split_year], X[X.Year > split_year]
yTr, yTe = y.loc[xTr.index, :], y.loc[xTe.index, :]

### 3.2 Normalization <br>
Following our various feature transformations and development, we arrived to a sparse dataframe with the exception of a few features(*Year, DayofYear*). It will be important to *normalize* these features as they are in gross magnitudes compared to the remaining features. During model training, having dominating features (in scale relative to others) can be dangerous as the weight updates may mistakengly favor these larger-scale features because it will have the largest influence on the target output. 

In [45]:
# minmax_scaler.fit_transform()：这个方法首先拟合数据，即计算数据的最小值和最大值，这些值用于后续的缩放。然后，它将这些参数用于转换数据，将原始数据缩放到0和1之间。
# minmax_scaler.transform()：这个方法使用在训练数据上计算得到的最小值和最大值来转换测试数据。这确保了训练数据和测试数据使用相同的缩放标准。
from sklearn.preprocessing import MinMaxScaler
minmax_scaler = MinMaxScaler()
xTr.loc[:, ['Year']] = minmax_scaler.fit_transform(xTr.loc[:, ['Year']])
xTe.loc[:, ['Year']] = minmax_scaler.transform(xTe.loc[:, ['Year']])

In [46]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
to_scale = ['HomeWinStreak','AwayWinStreak','HomeWinsToDate', 'AwayWinsToDate', 'HomeTeamELO', 'AwayTeamELO', 'HomeOdds', 'AwayOdds', 'DrawOdds'] + betting_feats

xTr.loc[:, to_scale] = std_scaler.fit_transform(xTr.loc[:, to_scale])
xTe.loc[:, to_scale] = std_scaler.transform(xTe.loc[:, to_scale])

In [47]:
xTr

,HomeOdds,AwayOdds,DrawOdds,B365H,B365D,B365A,IWH,IWD,IWA,WHH,...,Year,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak,HomeWinsToDate,AwayWinsToDate
0,0.003620,-0.548274,-0.718048,-0.044454,-0.763769,-0.540337,0.093413,-0.865960,-0.537905,-0.024579,...,0.0,-8.660254e-01,-0.5,-5.307300e-01,-0.847541,3,-0.374524,-0.238784,-1.278244,-1.04016
1,-0.223309,-0.405188,-0.677646,-0.274054,-0.657016,-0.383581,-0.181042,-0.865960,-0.367981,-0.203776,...,0.0,-8.660254e-01,-0.5,-5.307300e-01,-0.847541,3,-0.374524,-0.238784,-1.278244,-0.626407
2,-0.192364,-0.434399,-0.677646,-0.187954,-0.657016,-0.472182,-0.181042,-0.758080,-0.401966,-0.203776,...,0.0,-8.660254e-01,-0.5,-5.307300e-01,-0.847541,3,-0.374524,-0.238784,-1.278244,-0.626407
3,-0.402170,-0.184370,-0.645324,-0.427312,-0.657016,-0.165487,-0.386882,-0.758080,-0.130088,-0.382973,...,0.0,-8.660254e-01,-0.5,-5.307300e-01,-0.847541,3,-0.374524,-0.238784,-0.969979,-1.04016
4,-0.380302,-0.233881,-0.613002,-0.383688,-0.657016,-0.233641,-0.318269,-0.758080,-0.266027,-0.424786,...,0.0,-8.660254e-01,-0.5,-5.307300e-01,-0.847541,3,-0.374524,-0.238784,-1.278244,-0.626407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31297,4.831018,-0.935943,2.102040,4.260547,2.332082,-0.915187,5.170816,2.046802,-0.989903,5.052667,...,1.0,-2.449294e-16,1.0,6.432491e-16,1.000000,0,-0.374524,-0.238784,-0.969979,3.924882
31298,-0.182049,-0.327951,-0.419071,-0.216654,-0.400808,-0.329058,-0.146735,-0.542320,-0.317004,-0.173910,...,1.0,-2.449294e-16,1.0,6.432491e-16,1.000000,1,0.392761,-0.238784,1.804408,-0.626407
31299,5.243616,-0.935943,2.102040,5.408548,1.905068,-0.915187,5.170816,2.586203,-0.989903,5.052667,...,1.0,-2.449294e-16,1.0,6.432491e-16,1.000000,0,-0.374524,-0.238784,0.263082,2.269868
31300,0.282124,-0.560652,-0.580680,0.242546,-0.486210,-0.574414,0.333560,-0.758080,-0.554898,0.274082,...,1.0,-2.449294e-16,1.0,1.721336e-02,0.999852,2,-0.374524,-0.238784,0.879613,1.856114


In [48]:
xTe

,HomeOdds,AwayOdds,DrawOdds,B365H,B365D,B365A,IWH,IWD,IWA,WHH,...,Year,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak,HomeWinsToDate,AwayWinsToDate
31302,-0.243939,-0.278440,-0.241300,-0.274054,-0.230002,-0.301796,-0.215348,-0.272619,-0.249035,-0.233642,...,1.055556,0.5,0.866025,0.017213,0.999852,2,-0.374524,-0.238784,-1.278244,-0.626407
31303,-0.487372,-0.025936,0.356656,-0.492174,0.367817,-0.110963,-0.489803,0.320721,0.073821,-0.472571,...,1.055556,0.5,0.866025,0.017213,0.999852,1,-0.374524,-0.238784,-0.969979,-1.04016
31304,0.870076,-0.732949,-0.241300,0.759146,-0.230002,-0.724354,0.985389,-0.488379,-0.741814,0.871406,...,1.055556,0.5,0.866025,0.034422,0.999407,0,-0.374524,-0.238784,-1.278244,-1.04016
31305,2.107871,-0.854745,0.495640,1.821046,0.478841,-0.838854,2.597808,0.644361,-0.901543,1.946587,...,1.055556,0.5,0.866025,0.034422,0.999407,0,-0.374524,-0.238784,-1.278244,-0.626407
31306,0.069636,-0.481434,-0.564519,0.024426,-0.443509,-0.506259,0.093413,-0.758080,-0.452943,0.094885,...,1.055556,0.5,0.866025,0.034422,0.999407,1,-0.374524,-0.238784,-1.278244,-1.04016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36845,-0.192364,-0.298245,-0.483714,-0.187954,-0.571613,-0.301796,-0.146735,-0.434439,-0.283019,-0.233642,...,1.222222,0.5,0.866025,0.051620,0.998667,1,-0.374524,-0.238784,-0.969979,-1.04016
36846,-0.817450,2.746664,2.586869,-0.790654,2.332082,2.833315,-0.853455,3.125603,2.996513,-0.801099,...,1.222222,0.5,0.866025,0.051620,0.998667,2,-0.374524,-0.238784,-0.969979,-1.04016
36847,1.839682,-0.850784,0.592606,1.677546,0.478841,-0.827949,2.083206,0.752241,-0.887949,1.767390,...,1.222222,0.5,0.866025,0.068802,0.997630,1,-0.374524,-0.238784,-1.278244,-0.626407
36848,-0.522442,0.087939,0.259690,-0.503654,0.197012,0.038977,-0.524109,0.320721,0.209760,-0.532304,...,1.222222,0.5,0.866025,0.068802,0.997630,2,-0.374524,-0.238784,-0.969979,-1.04016


### 3.3 HomeWins Baseline Model

In [49]:
from sklearn.metrics import accuracy_score

In [50]:
xTr.shape

(31302, 437)

In [51]:
xTe.shape

(5548, 437)

In [52]:
# training score
baseline_Tr = np.full((xTr.shape[0], 1), 2) 
accuracy_score(yTr.Result.values, baseline_Tr.ravel())

0.46003450258769407

In [53]:
# testing score
baseline_preds_Te = np.full((xTe.shape[0]  , 1), 2) #predicts home wins all the time
accuracy_score(yTe.Result.values, baseline_preds_Te.ravel())

0.4396178803172314

### 3.4 Multinomial Logistic Regression

**3.4.1** $l2$ Regularized

In [54]:
from sklearn.linear_model import LogisticRegression
l2_lr = LogisticRegression(max_iter = 10000, n_jobs=-1).fit(xTr, yTr.values.ravel())

In [55]:
# training score
accuracy_score(yTr.Result.values, l2_lr.predict(xTr))

0.641332822183886

In [56]:
# testing score
lr_preds = l2_lr.predict(xTe)
accuracy_score(yTe.Result.values, lr_preds)

0.6169790915645278

**3.4.1** $l2$ Penalty Tuning

In [57]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

logistic_params = {'C':[0.001,0.01,0.10]}

# logistic_randsearch = RandomizedSearchCV(estimator=LogisticRegression(max_iter=10000),
#                                          param_distributions=logistic_params,
logistic_randsearch = GridSearchCV(estimator=LogisticRegression(max_iter=10000),
                                         param_grid=logistic_params,
                                         scoring='accuracy',
                                         verbose=1,
                                         cv=5,
                                         n_jobs=-1)

logistic_rand_results = logistic_randsearch.fit(xTr, yTr.values.ravel())
print("Best: %f using %s" % (logistic_rand_results.best_score_, logistic_rand_results.best_params_))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best: 0.623443 using {'C': 0.01}


In [58]:
l2_rs = logistic_rand_results.best_estimator_

In [59]:
# training score
accuracy_score(yTr.Result.values, l2_rs.predict(xTr))

0.6335697399527187

In [60]:
# testing score
accuracy_score(yTe.Result.values, l2_rs.predict(xTe))

0.6276135544340303

**3.4.4** $l1$ Regularized

In [61]:
l1_lr = LogisticRegression(penalty='l1', solver='saga', max_iter = 10000, n_jobs=-1).fit(xTr, yTr.values.ravel())

In [62]:
# training score
accuracy_score(yTr.Result.values, l1_lr.predict(xTr))

0.6403424701297041

In [63]:
# testing score
l1_preds = l1_lr.predict(xTe)
accuracy_score(yTe.Result.values, l1_preds)

0.6180605623648161

**3.4.5** Penalty Tuning

In [64]:
l1_params = {'C':[0.001,0.01,0.10]}

# l1_randsearch = RandomizedSearchCV(estimator=LogisticRegression(penalty='l1',solver='saga', max_iter=10000),
#                                          param_distributions=l1_params,
l1_randsearch = GridSearchCV(estimator=LogisticRegression(penalty='l1',solver='saga', max_iter=10000),
                                         param_grid=l1_params,
                                         scoring='accuracy',
                                         verbose=1,
                                         n_jobs=-1,
                                         cv=5)

l1_rand_results = l1_randsearch.fit(xTr, yTr.values.ravel())
print("Best: %f using %s" % (l1_rand_results.best_score_, l1_rand_results.best_params_))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best: 0.623954 using {'C': 0.1}


In [65]:
l1_rs = l1_randsearch.best_estimator_ #LogisticRegression(C=0.10, solver='saga', max_iter=10000).fit(xTr, yTr.values.ravel())#

In [66]:
# training score
accuracy_score(yTr.Result.values, l1_rs.predict(xTr))

0.6311737269184078

In [67]:
# testing score
accuracy_score(yTe.Result.values, l1_rs.predict(xTe))

0.6308579668348955

### 3.5 Support Vector Machine

In [68]:
from sklearn.svm import SVC
svm = SVC(max_iter=100000).fit(xTr, yTr.values.ravel())

In [69]:
# training score
accuracy_score(yTr.Result.values, svm.predict(xTr))

0.6905948501693182

In [70]:
# testing score
accuracy_score(yTe.Result.values, svm.predict(xTe))

0.647801009372747

**3.5.2** Penalty Tuning

In [71]:
svm_params = {'C':[0.001,0.01,0.10]}

# svm_randsearch = RandomizedSearchCV(estimator=SVC(max_iter=100000),
#                                          param_distributions=svm_params,
svm_randsearch = GridSearchCV(estimator=SVC(max_iter=100000),
                                         param_grid=svm_params,
                                         scoring='accuracy',
                                         verbose=2,
                                         cv=5,
                                         n_jobs=-1)

svm_rand_results = svm_randsearch.fit(xTr, yTr.values.ravel())
print("Best: %f using %s" % (svm_rand_results.best_score_, svm_rand_results.best_params_))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best: 0.619833 using {'C': 0.1}


In [72]:
svm_rs = svm_rand_results.best_estimator_

In [73]:
# training score
accuracy_score(yTr.Result.values, svm_rs.predict(xTr))

0.6319404510893872

In [74]:
# testing score
accuracy_score(yTe.Result.values, svm_rs.predict(xTe))

0.6297764960346071

### 3.6 Simple Neural Network ####

In [75]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(512,128,32),
                    activation='relu',
                    batch_size=64,
                    max_iter=200,
                    learning_rate_init=1e-4,
                    early_stopping=False,
                    alpha=1e-3,
                   ).fit(xTr, yTr.values.ravel())

In [76]:
# training score
accuracy_score(yTr.Result.values, mlp.predict(xTr))

0.9984346048175835

In [77]:
# testing score
accuracy_score(yTe.Result.values, mlp.predict(xTe))

0.6353640951694304

### 3.7 Stacked Classifier ###

In [78]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [79]:
from sklearn.ensemble import StackingClassifier
stacked_clf = StackingClassifier(estimators=[('svm', SVC(max_iter=100000)), ('logistic', LogisticRegression(C=0.01, max_iter=10000))],
                                final_estimator=LogisticRegression(max_iter=10000),
                                n_jobs=-1).fit(xTr, yTr.values.ravel())

In [80]:
# training score
accuracy_score(yTr.Result.values, stacked_clf.predict(xTr))

0.655676953549294

In [81]:
# testing score
accuracy_score(yTe.Result.values, stacked_clf.predict(xTe))

0.6398702235039654

## 4. Result Analysis ##

In [82]:
## TODO: breakdown results across divisions and/or teams; i.e., see how model performs individually at subgroups

## 5. Scrap Code ##

In [83]:
barcelona_df = learning_df[(learning_df['HomeTeam 17'] == 1) | (learning_df['AwayTeam 17'] == 1)]
barcelona_df

,HomeOdds,AwayOdds,DrawOdds,B365H,B365D,B365A,IWH,IWD,IWA,WHH,...,Year,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak,HomeWinsToDate,AwayWinsToDate
102,1.526667,5.433333,3.433333,1.500,3.40,6.00,1.55,3.50,4.80,1.53,...,2002,-1.000000e+00,-1.836970e-16,-0.871706,-0.490029,3,0,0,0,0
142,2.950000,2.200000,3.033333,3.000,3.00,2.20,3.10,3.00,2.10,2.75,...,2002,-1.000000e+00,-1.836970e-16,-0.958718,-0.284359,3,0,0,0,1
189,1.331333,7.666667,4.083333,1.364,3.75,8.00,1.35,4.00,7.00,1.28,...,2002,-1.000000e+00,-1.836970e-16,-0.985948,-0.167052,3,0,0,1,0
236,2.583333,2.400000,3.116667,2.750,3.25,2.20,2.50,3.00,2.50,2.50,...,2002,-1.000000e+00,-1.836970e-16,-0.998880,-0.047321,3,0,0,1,1
302,1.245333,9.333333,4.666667,1.286,4.50,8.00,1.20,5.00,10.00,1.25,...,2002,-8.660254e-01,5.000000e-01,-0.995919,0.090252,3,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36638,1.983333,3.516667,3.700000,2.000,3.75,3.50,2.00,3.65,3.55,1.95,...,2023,-2.449294e-16,1.000000e+00,-0.463550,0.886071,2,1,0,16,8
36697,1.500000,5.500000,4.816667,1.530,5.00,5.50,1.53,4.70,5.50,1.44,...,2023,-2.449294e-16,1.000000e+00,-0.353676,0.935368,1,0,6,16,9
36740,4.933333,1.653333,3.916667,5.000,3.80,1.67,5.00,3.95,1.67,4.80,...,2023,-2.449294e-16,1.000000e+00,-0.255353,0.966848,0,0,0,8,10
36772,1.130000,16.000000,8.750000,1.140,8.50,15.00,1.15,8.75,16.00,1.10,...,2023,-2.449294e-16,1.000000e+00,-0.188227,0.982126,2,0,0,17,1


In [84]:
bxTr = xTr[(xTr['HomeTeam 17'] == 1) | (xTr['AwayTeam 17'] == 1)]
bxTe = xTe[(xTe['HomeTeam 17'] == 1) | (xTe['AwayTeam 17'] == 1)]

In [85]:
byTr, byTe = yTr.loc[bxTr.index,:], yTe.loc[bxTe.index,:]

In [86]:
# training score
accuracy_score(byTr, l1_lr.predict(bxTr))

0.7411242603550295

In [87]:
# testing score
accuracy_score(byTe, l1_lr.predict(bxTe))

0.6949152542372882

In [88]:
# training score
accuracy_score(byTr, l2_lr.predict(bxTr))

0.735207100591716

In [89]:
# testing score
accuracy_score(byTe, l2_lr.predict(bxTe))

0.6949152542372882

## 6. Pytorch MLP ##

In [90]:
type(xTr)

pandas.core.frame.DataFrame

In [91]:
xTr.shape

(31302, 437)

In [92]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cuda


In [93]:
class Attention(nn.Module):
    def __init__(self, feature_dim):
        super(Attention, self).__init__()
        self.attention_weights = nn.Parameter(torch.randn(feature_dim))

    def forward(self, x):
        # 应用注意力权重
        weights = F.softmax(self.attention_weights, dim=0)
        # 加权求和
        x = x * weights
        return x
    
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(in_features=xTr.shape[1], out_features=512)
        self.bn1 = nn.BatchNorm1d(num_features=512)
        self.attention = Attention(512)
        self.dropout1 = nn.Dropout(p=0.2)
        
        self.fc2 = nn.Linear(in_features=512, out_features=128)
        self.bn2 = nn.BatchNorm1d(num_features=128)
        self.dropout2 = nn.Dropout(p=0.2)
        
        self.fc3 = nn.Linear(in_features=128, out_features=32)
        self.bn3 = nn.BatchNorm1d(num_features=32)
        self.dropout3 = nn.Dropout(p=0.2)
        
        self.fc4 = nn.Linear(in_features=32, out_features=3)  # 输出层改为3，对应三个类别

    def forward(self, x):
        x = self.dropout1(torch.relu(self.bn1(self.fc1(x))))
        x = self.attention(x)
        x = self.dropout2(torch.relu(self.bn2(self.fc2(x))))
        x = self.dropout3(torch.relu(self.bn3(self.fc3(x))))
        x = self.fc4(x)
        return x

# 数据预处理
scaler = StandardScaler()
xTr_scaled = scaler.fit_transform(xTr)
xTr_tensor = torch.tensor(xTr_scaled, dtype=torch.float32).to(device)
yTr_tensor = torch.tensor(yTr.values.ravel(), dtype=torch.long).to(device)

# 创建数据加载器
dataset = TensorDataset(xTr_tensor, yTr_tensor)
dataloader = DataLoader(dataset, batch_size=1024, shuffle=True)

# 创建模型实例
model = MLP().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

train_start = time.time()
# 训练模型
model.train()
for epoch in range(500):  # 假设训练200个epoch
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # 清除之前的梯度

        outputs = model(inputs)  # 前向传播
        loss = criterion(outputs, labels)  # 计算损失
        loss.backward()  # 反向传播
        optimizer.step()  # 更新参数

        running_loss += loss.item() * inputs.size(0)  # 累计损失
        _, predicted = torch.max(outputs.data, 1)  # 获取预测结果
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total * 100  # 计算准确率

    # 每个epoch结束后输出
    print(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')
print(f'训练时长： {time.time() - train_start}s')

Epoch 1, Loss: 1.2067, Accuracy: 29.54%
Epoch 2, Loss: 1.1333, Accuracy: 34.71%
Epoch 3, Loss: 1.0792, Accuracy: 39.91%
Epoch 4, Loss: 1.0382, Accuracy: 43.80%
Epoch 5, Loss: 1.0099, Accuracy: 47.05%
Epoch 6, Loss: 0.9816, Accuracy: 49.91%
Epoch 7, Loss: 0.9593, Accuracy: 52.44%
Epoch 8, Loss: 0.9394, Accuracy: 53.81%
Epoch 9, Loss: 0.9187, Accuracy: 55.39%
Epoch 10, Loss: 0.9001, Accuracy: 56.82%
Epoch 11, Loss: 0.8813, Accuracy: 57.70%
Epoch 12, Loss: 0.8642, Accuracy: 58.73%
Epoch 13, Loss: 0.8510, Accuracy: 59.66%
Epoch 14, Loss: 0.8356, Accuracy: 60.74%
Epoch 15, Loss: 0.8262, Accuracy: 61.08%
Epoch 16, Loss: 0.8137, Accuracy: 61.73%
Epoch 17, Loss: 0.8051, Accuracy: 62.26%
Epoch 18, Loss: 0.7925, Accuracy: 63.04%
Epoch 19, Loss: 0.7834, Accuracy: 63.42%
Epoch 20, Loss: 0.7759, Accuracy: 63.77%
Epoch 21, Loss: 0.7675, Accuracy: 64.22%
Epoch 22, Loss: 0.7597, Accuracy: 64.41%
Epoch 23, Loss: 0.7521, Accuracy: 64.59%
Epoch 24, Loss: 0.7449, Accuracy: 65.07%
Epoch 25, Loss: 0.7368, A

In [94]:
# 假设 xTe 和 yTe 是 pandas DataFrame 或 Series
# 数据预处理
xTe_scaled = scaler.fit_transform(xTe)  # 使用与训练数据相同的标准化参数
xTe_tensor = torch.tensor(xTe_scaled, dtype=torch.float32).to(device)
yTe_tensor = torch.tensor(yTe.values.ravel(), dtype=torch.long).to(device)

# 创建数据加载器
test_dataset = TensorDataset(xTe_tensor, yTe_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 设置模型为评估模式
model.eval()

# 初始化用于计算准确率的变量
correct = 0
total = 0

# 不计算梯度，因为在评估模式下不需要进行反向传播
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# 计算准确率
accuracy = correct / total
print(f'Accuracy on test set: {accuracy * 100:.2f}%')

Accuracy on test set: 60.04%


## 7. Pytorch Transformer ##

In [95]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_classes, num_heads, num_layers, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.input_dim = input_dim
        self.num_classes = num_classes
        self.model_dim = input_dim  # 通常情况下，模型维度与输入维度相同

        # Transformer Encoder Layer
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=self.model_dim,
            nhead=num_heads,
            dim_feedforward=512,  # 前馈网络的维度
            dropout=dropout,
            batch_first=True
        )

        # Transformer Encoder
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # 输出层
        self.output_layer = nn.Linear(self.model_dim, self.num_classes)

        # Batch Normalization
        self.bn = nn.BatchNorm1d(self.model_dim)

    def forward(self, x):
        # 增加一个假的序列维度
        x = x.unsqueeze(1)
        # Transformer Encoder
        x = self.transformer_encoder(x)

        # Batch Normalization
        x = self.bn(x[:, 0, :])  # 取序列的第一个元素进行批量归一化

        # 输出层
        x = self.output_layer(x)
        return x

# 补充维度
n_samples_xTr = xTr.shape[0]
n_samples_xTe = xTe.shape[0]
for i in range(1, 4):  # 从 1 到 3，因为需要添加三列
    xTr[f'pad{i}'] = 0  # 添加填充列，初始化为 0
    xTe[f'pad{i}'] = 0  # 添加填充列，初始化为 0

# 参数设置
input_dim = xTr.shape[1]  # 输入特征的维度
num_classes = 3  # 类别数
num_heads = 10  # 注意力头的数量
num_layers = 3  # Transformer层的数量
dropout = 0.8  # Dropout比率

# 创建模型
model = TransformerModel(input_dim, num_classes, num_heads, num_layers, dropout).to(device)

# 损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# 数据加载
# 假设 xTr 和 xTe 已经是适当的 torch.Tensor 对象
xTr_values = xTr.values.astype(float)
xTe_values = xTe.values.astype(float)

xTr_tensor = torch.tensor(xTr_values, dtype=torch.float32).to(device)
xTe_tensor = torch.tensor(xTe_values, dtype=torch.float32).to(device)
yTr_tensor = torch.tensor(yTr.values, dtype=torch.long).to(device).squeeze(1)
yTe_tensor = torch.tensor(yTe.values, dtype=torch.long).to(device).squeeze(1)
# 转换为 one-hot 编码
yTr_tensor = F.one_hot(yTr_tensor, num_classes=num_classes).float()
yTe_tensor = F.one_hot(yTe_tensor, num_classes=num_classes).float()

# 数据加载器
train_dataset = TensorDataset(xTr_tensor, yTr_tensor)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=False)

test_dataset = TensorDataset(xTe_tensor, yTe_tensor)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

train_start = time.time()
# 训练模型
model.train()
for epoch in range(500):  # 运行更多的 epoch 以获得更好的结果
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)  # 累计损失
        _, predicted = torch.max(outputs.data, 1)  # 获取预测结果
        _, truth = torch.max(labels.data, 1)
        total += truth.size(0)
        correct += (predicted == truth).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total * 100  # 计算准确率

    # 每个epoch结束后输出
    print(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')
print(f'训练时长： {time.time() - train_start}s')

# 设置模型为评估模式
model.eval()

# 初始化用于计算准确率的变量
correct = 0
total = 0

# 不计算梯度，因为在评估模式下不需要进行反向传播
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        _, truth = torch.max(labels.data, 1)
        total += labels.size(0)
        correct += (predicted == truth).sum().item()

# 计算准确率
accuracy = correct / total
print(f'Accuracy on test set: {accuracy * 100:.2f}%')

D:\Anaconda3\Lib\site-packages\torch\nn\functional.py:5504: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


Epoch 1, Loss: 1.1752, Accuracy: 35.45%
Epoch 2, Loss: 1.1269, Accuracy: 39.19%
Epoch 3, Loss: 1.0799, Accuracy: 43.39%
Epoch 4, Loss: 1.0438, Accuracy: 47.50%
Epoch 5, Loss: 1.0121, Accuracy: 50.40%
Epoch 6, Loss: 0.9866, Accuracy: 52.53%
Epoch 7, Loss: 0.9632, Accuracy: 53.99%
Epoch 8, Loss: 0.9437, Accuracy: 55.19%
Epoch 9, Loss: 0.9258, Accuracy: 56.33%
Epoch 10, Loss: 0.9133, Accuracy: 56.54%
Epoch 11, Loss: 0.9027, Accuracy: 57.50%
Epoch 12, Loss: 0.8912, Accuracy: 57.92%
Epoch 13, Loss: 0.8834, Accuracy: 57.82%
Epoch 14, Loss: 0.8771, Accuracy: 58.15%
Epoch 15, Loss: 0.8711, Accuracy: 58.27%
Epoch 16, Loss: 0.8657, Accuracy: 58.65%
Epoch 17, Loss: 0.8590, Accuracy: 58.77%
Epoch 18, Loss: 0.8528, Accuracy: 58.99%
Epoch 19, Loss: 0.8481, Accuracy: 58.99%
Epoch 20, Loss: 0.8470, Accuracy: 58.99%
Epoch 21, Loss: 0.8394, Accuracy: 59.01%
Epoch 22, Loss: 0.8319, Accuracy: 59.54%
Epoch 23, Loss: 0.8280, Accuracy: 59.47%
Epoch 24, Loss: 0.8234, Accuracy: 59.39%
Epoch 25, Loss: 0.8176, A